# Extracting the zip file.

In [0]:
import zipfile
with zipfile.ZipFile("/content/Language_1.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/Language")

**Importing necessary libraries**

In [0]:
import pandas as pd
import numpy as np

#Labels 
**1 denotes positive sentiment and 0 denotes negative sentiment.**

**reading files from amazon.txt**

In [0]:
df1 = pd.read_csv("/content/Language/Language_1/Round1_Problem1-of-2_Dataset_amazon_cells_labelled.txt", header=None, delimiter="\t", names=["text", "sentiment"])
df1.shape

(1000, 2)

In [0]:
df1.head(5)

,text,sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


**reading files from imdb.txt**

In [0]:
df2 = pd.read_csv("/content/Language/Language_1/Round1_Problem1-of-2_Dataset_imdb_labelled.txt", header=None, delimiter="\t",names=["text", "sentiment"])
df2.shape

(748, 2)

In [0]:
df2.head(5)

,text,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


# Merging both the dataframe as one with text and sentiment.

In [0]:
frames = [df1, df2]
df = pd.concat(frames, axis =0)
df.shape

(1748, 2)

In [0]:
df.head(5)

,text,sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


**visualizing 5 full list values**

In [0]:
df.text.tolist()[1:5]

['Good case, Excellent value.',
 'Great for the jawbone.',
 'Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!',
 'The mic is great.']

# removing html

In [0]:
#Regex (Regualar Expr Operations)
import re 

#Function to remove html tags from data
def removehtml(data):
    p = re.compile('<.*?>')#Find this kind of pattern
    #print(p.findall(data))#List of strings which follow the regex pattern
    return p.sub('',data) #Substitute nothing at the place of strings which matched the patterns

#Checking with my own html if this works or not  
removehtml('<a href="foo.com" class="bar">My name is Navneet <b>, Hello!</b></a><>')

'My name is Navneet , Hello!'

**removing all html from text**

In [0]:
text_without_html = []
for i in df.text.values:
  sentence = removehtml(i)
  text_without_html.append(sentence)
  
df.text = text_without_html  

In [0]:
text_without_html[975:980]

['It is the best charger I have seen on the market yet.',
 'SWEETEST PHONE!!!',
 ':-)Oh, the charger seems to work fine.',
 'It fits so securely that the ear hook does not even need to be used and the sound is better directed through your ear canal.',
 'Not enough volume.']

# Remove website links

**checking with my own strings if this works or not and hopefully it worked**

In [0]:
lis = ["hello www.xyz.com"]
f = pd.DataFrame(lis, columns=["one"])
f = f.one.str.replace("www.\S+", " ", case=False)
f

0    hello  
Name: one, dtype: object

**removing html links from text**

In [0]:
df['text'] = df['text'].str.replace('www.\S+', '', case=False)

# Convert	major	emoticons	to	their	respective	emotions	

**checking two rows with smily faces**

In [0]:
print(df.text[387])
print(df.text[977])

387    The best phone in market :).
387              Not recommended.  
Name: text, dtype: object
:-)Oh, the charger seems to work fine.


**converting smily faces with replace**

In [0]:
df["text"] = df["text"].replace(["\:\)", "\:\-\)", "\:\-\}", "\;\-\}", "\:\-\>", "\;\-\)"], ["Happy","Happy","Happy","Happy","Happy","Happy"], regex=True)
df["text"] = df["text"].replace(["\:\-\(", "\:\(", "\:\-\|", "\;\-\(", "\;\-\<", "\|\-\{"], ["Sad", "Sad", "Sad", "Sad", "Sad", "Sad",], regex=True)
df["text"] = df["text"].replace(["\:\D", "\:\'\-\)", "\:\`\-\("], ["laugh", "tear of joy", "tear of sadness"], regex=True)


**we can see below that all smiley faces have been repaced by therir respective meaning**

In [0]:
print(df.text[387])
print(df.text[977])



387    The best phone in market Happy.
387                 Not recommended.  
Name: text, dtype: object
HappyOh, the charger seems to work fine.


# Removing all punctuation except few


In [0]:
df["text"].tolist()[14]

'The design is very odd, as the ear "clip" is not very comfortable at all.'

**here final represents all punctuations except . ! and ,**

In [0]:
import string
exclude = set(string.punctuation)
not_exclude={".", "!", ","}
final = exclude-not_exclude

print(len(exclude))
print(len(not_exclude))
print(len(final))

32
3
29


In [0]:
for i in final:
  df['text'] = df['text'].str.replace(i, '', regex=True)

**we can see that punctuation "clip " is replaced by clip.**

In [0]:
df["text"].tolist()[14]

'The design is very odd, as the ear clip is not very comfortable at all.'

# Removing major stop words

**downloading stopwords**

In [0]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**extending stopwords with nltk_words to remove maximum unnecessary words**

In [0]:
#!pip3 install stop_words

In [0]:
#https://stackoverflow.com/questions/5486337/how-to-remove-stop-words-using-nltk-or-python
from stop_words import get_stop_words
from nltk.corpus import stopwords

stop_words = list(get_stop_words('en'))         #About 900 stopwords
nltk_words = list(stopwords.words('english')) #About 179 stopwords
stop_words.extend(nltk_words)

len(stop_words)

353

In [0]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_text = []
# tqdm is for printing the status bar
for sentance in tqdm(df['text'].values):
    sent = ' '.join(e for e in sentance.split() if e not in stop_words)
    preprocessed_text.append(sent.lower().strip())

df["text"] = preprocessed_text    

100%|██████████| 1748/1748 [00:00<00:00, 18149.38it/s]


**below we can see that all the stop words have been removed**

In [0]:
df.text.tolist()[5:10]

['i jiggle plug get line right get decent volume.',
 'if several dozen several hundred contacts, imagine fun sending one one.',
 'if razr owner...you must this!',
 'needless say, i wasted money.',
 'what waste money time!.']

# Lemmatize	all	the	words.

In [0]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 

lemet_text = []
# tqdm is for printing the status bar
for sentance in tqdm(df['text'].values):
    sent = ' '.join(e for e in sentance.split() if e in stemmer.stem(e) )
    lemet_text.append(sent.lower().strip())

df["text"] = lemet_text    


100%|██████████| 1748/1748 [00:00<00:00, 7083.25it/s]


**we can see below that all words are in their lematized form**

In [0]:
df.text.tolist()[5:10]

['i plug get line right get decent volume.',
 'if dozen contacts, fun one one.',
 'if razr owner...you must this!',
 'needless say, i money.',
 'what money time!.']

# Tokenize	the	data

In [0]:
import nltk
nltk.download('punkt')

df['text'] = df.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
df.text.tolist()[5:10]

[['i', 'plug', 'get', 'line', 'right', 'get', 'decent', 'volume', '.'],
 ['if', 'dozen', 'contacts', ',', 'fun', 'one', 'one', '.'],
 ['if', 'razr', 'owner', '...', 'you', 'must', 'this', '!'],
 ['needless', 'say', ',', 'i', 'money', '.'],
 ['what', 'money', 'time', '!', '.']]

# converting	the	data	into	vector	forms	using	Word2Vec	with	vector	size	of	20

In [0]:
from gensim.models import Word2Vec
# train model
model = Word2Vec(df.text.values, min_count=1, size = 20)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(words)
# access vector for one word
print(model['plug'])
# save model
model.save('model.bin')
# load model
new_model = Word2Vec.load('model.bin')
print(new_model)

Word2Vec(vocab=2674, size=20, alpha=0.025)
['so', 'way', 'plug', 'us', 'unless', 'i', 'go', 'converter', '.', 'good', 'case', ',', 'value', 'great', 'jawbone', 'charger', '45', 'minutes.major', 'problems', '!', 'the', 'mic', 'get', 'line', 'right', 'decent', 'volume', 'if', 'dozen', 'contacts', 'fun', 'one', 'razr', 'owner', '...', 'you', 'must', 'this', 'needless', 'say', 'money', 'what', 'time', 'and', 'sound', 'he', 'battery', 'two', 'mere', '5', 'ft', 'static', 'headset', 'though', 'design', 'odd', 'ear', 'clip', 'all', 'recommend', 'blue', 'tooth', 'phone', 'do', 'not', 'be', 'fooled', 'far', 'it', 'place', 'wonder', 'long', 'last', 'went', 'directions', 'pair', 'again', 'bought', 'use', 'fire', 'misleading', 'yet', 'run', 'new', 'three', 'without', 'charging', 'mother', 'problem', 'pocket', 'pc', 'combination', 'ive', '7', 'best', 'had', 'didnt', 'think', 'me', 'couldnt', 'hear', 'talk', 'pull', 'doesnt', 'hold', 'charge', 'unacceptible', 'product', 'ideal', 'like', 'whose', 'sen

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
print(model['you'])

[ 0.10848033  0.12167265 -0.0458592  -0.20120674 -0.03437223 -0.20311394
  0.22664073 -0.03371485  0.30419353  0.00376729  0.00348562  0.07483245
 -0.41990697 -0.04048717 -0.27936962 -0.17079318  0.21314475 -0.29875755
 -0.18865399 -0.29240727]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [0]:
df.to_csv("/content/output.csv")

In [0]:
df1= pd.read_csv("output.csv")
df1.head(4)

,Unnamed: 0,text,sentiment
0,0,"['so', 'way', 'plug', 'us', 'unless', 'i', 'go...",0
1,1,"['good', 'case', ',', 'value', '.']",1
2,2,"['great', 'jawbone', '.']",1
3,3,"['charger', '45', 'minutes.major', 'problems',...",0
